In [5]:
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Flatten,Activation,Conv1D,MaxPooling1D,Dropout,LSTM, Bidirectional, GlobalMaxPooling1D,LeakyReLU
from tensorflow.keras.layers import Embedding

In [6]:
df= pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
df.drop_duplicates(inplace=True)
df.shape

(49582, 2)

In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
df["review"][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

In [13]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('punkt') # Used for sentence tokenizer
nltk.download('stopwords')
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from numpy import array

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
def review_preprocess(review):
    """
    Takes in a string of review, then performs the following:
    1. Remove HTML tag from review
    2. Remove URLs from review
    3. Make entire review lowercase
    4. Split the review in words
    5. Remove all punctuation
    6. Remove empty strings from review
    7. Remove all stopwords
    8. Returns a list of the cleaned review after jioning them back to a sentence
    """
    en_stops = set(stopwords.words('english'))
    
    """
    Removing HTML tag from review
    """
    clean = re.compile('<.*?>')
    review_without_tag = re.sub(clean, '', review) 
    
    
    """
    Removing URLs
    """
    review_without_tag_and_url = re.sub(r"http\S+", "", review_without_tag)
    
    review_without_tag_and_url = re.sub(r"www\S+", "", review_without_tag)
    
    """
    Make entire string lowercase
    """
    review_lowercase = review_without_tag_and_url.lower()
    
    """
    Split string into words
    """
    list_of_words = word_tokenize(review_lowercase)
    
    
    """
    Remove punctuation
    Checking characters to see if they are in punctuation
    """

    list_of_words_without_punctuation=[''.join(this_char for this_char in this_string if (this_char in string.ascii_lowercase))for this_string in list_of_words]
     
    
    """
    Remove empty strings
    """
    list_of_words_without_punctuation = list(filter(None, list_of_words_without_punctuation))
    
    
    """
    Remove any stopwords
    """
  
    filtered_word_list = [w for w in list_of_words_without_punctuation if w not in en_stops] 
    
    """
    Returns a list of the cleaned review after jioning them back to a sentence
    """
    return ' '.join(filtered_word_list)

In [15]:
df['review']=df['review'].apply(review_preprocess)

In [16]:
df['review'].head()

0    one reviewers mentioned watching oz episode ho...
1    wonderful little production filming technique ...
2    thought wonderful way spend time hot summer we...
3    basically family little boy jake thinks zombie...
4    petter mattei love time money visually stunnin...
Name: review, dtype: object

In [18]:
y = np.array([1 if sentiment == 'positive' else 0 for sentiment in df['sentiment']])
y

array([1, 1, 1, ..., 0, 0, 0])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.3, random_state=42)

In [22]:
X_train.shape

(34707,)

In [23]:
y_test.shape

(14875,)

In [24]:
y_train = np.array([1 if sentiment == 'positive' else 0 for sentiment in y_train])
y_test = np.array([1 if sentiment == 'positive' else 0 for sentiment in y_test])

In [25]:
y_test.shape

(14875,)

In [27]:
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer

In [30]:
model = Word2Vec(df['review'], vector_size=100, window=5, min_count=1, workers=4)

In [33]:
df = df.reset_index()

In [34]:
def avgWord2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

In [36]:
from tqdm import tqdm
x = []
for i in tqdm(range(len(df['review']))):
    #print("Applying avg word2vec",i)
    x.append(avgWord2vec(df['review'][i]))

100%|██████████| 49582/49582 [01:18<00:00, 630.21it/s]


In [37]:
Xnew = np.array(x)
Xnew.shape

(49582, 100)

In [38]:
y = pd.get_dummies(df['sentiment'])
y = y.iloc[:,1].values
y

array([ True,  True,  True, ..., False, False, False])

In [39]:
X_train, X_test, y_train, y_test = train_test_split(Xnew,y,test_size=0.2,random_state=0)

In [41]:
from sklearn.naive_bayes import GaussianNB
model_analysis = GaussianNB().fit(X_train,y_train)
prediction = model_analysis.predict(X_test)

In [42]:
from sklearn.metrics import accuracy_score,classification_report as cr

score=accuracy_score(y_test,prediction)
print(score)
print(cr(y_test,prediction))

0.5823333669456489
              precision    recall  f1-score   support

       False       0.57      0.61      0.59      4909
        True       0.59      0.56      0.57      5008

    accuracy                           0.58      9917
   macro avg       0.58      0.58      0.58      9917
weighted avg       0.58      0.58      0.58      9917



In [43]:
from sklearn.ensemble import AdaBoostClassifier

model_analysis5 = AdaBoostClassifier().fit(X_train,y_train)
Prediction5 = model_analysis5.predict(X_test)
score=accuracy_score(y_test,Prediction5)
print(score)
print(cr(y_test,Prediction5))

0.5937279419179188
              precision    recall  f1-score   support

       False       0.59      0.59      0.59      4909
        True       0.60      0.60      0.60      5008

    accuracy                           0.59      9917
   macro avg       0.59      0.59      0.59      9917
weighted avg       0.59      0.59      0.59      9917



In [44]:
from sklearn.ensemble import RandomForestClassifier

model_analysis2 = RandomForestClassifier().fit(X_train,y_train)
Prediction2 = model_analysis2.predict(X_test)
score=accuracy_score(y_test,Prediction2)
print(score)
print(cr(y_test,Prediction2))

0.5955430069577493
              precision    recall  f1-score   support

       False       0.59      0.60      0.60      4909
        True       0.60      0.59      0.60      5008

    accuracy                           0.60      9917
   macro avg       0.60      0.60      0.60      9917
weighted avg       0.60      0.60      0.60      9917



In [51]:
from xgboost import XGBClassifier

model_analysis3 = XGBClassifier().fit(X_train,y_train)
Prediction3 = model_analysis3.predict(X_test)
score=accuracy_score(y_test,Prediction3)
print(score)
print(cr(y_test,Prediction3))

0.5833417364122214
              precision    recall  f1-score   support

       False       0.58      0.58      0.58      4909
        True       0.59      0.58      0.59      5008

    accuracy                           0.58      9917
   macro avg       0.58      0.58      0.58      9917
weighted avg       0.58      0.58      0.58      9917



In [45]:
model6 = Sequential()
model6.add(Dense(256, activation=LeakyReLU(), input_shape=(100,)))
model6.add(Dropout(0.2))
model6.add(Dense(128, activation=LeakyReLU()))
model6.add(Dropout(0.2))
model6.add(Dense(64, activation=LeakyReLU()))
model6.add(Dropout(0.2))
model6.add(Dense(1, activation='sigmoid'))

model6.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               25856     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6

In [46]:
model6.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model6.fit(X_train, y_train, epochs=10, batch_size=10, validation_data=(X_test, y_test))

Epoch 1/10
3967/3967 [==============================] - 8s 2ms/step - loss: 0.6796 - accuracy: 0.5630 - val_loss: 0.6678 - val_accuracy: 0.5922
Epoch 2/10
3967/3967 [==============================] - 7s 2ms/step - loss: 0.6708 - accuracy: 0.5888 - val_loss: 0.6701 - val_accuracy: 0.5901
Epoch 3/10
3967/3967 [==============================] - 7s 2ms/step - loss: 0.6691 - accuracy: 0.5935 - val_loss: 0.6659 - val_accuracy: 0.5991
Epoch 4/10
3967/3967 [==============================] - 7s 2ms/step - loss: 0.6675 - accuracy: 0.5978 - val_loss: 0.6673 - val_accuracy: 0.5943
Epoch 5/10
3967/3967 [==============================] - 7s 2ms/step - loss: 0.6664 - accuracy: 0.5986 - val_loss: 0.6883 - val_accuracy: 0.5639
Epoch 6/10
3967/3967 [==============================] - 7s 2ms/step - loss: 0.6661 - accuracy: 0.6003 - val_loss: 0.6623 - val_accuracy: 0.6025
Epoch 7/10
3967/3967 [==============================] - 7s 2ms/step - loss: 0.6644 - accuracy: 0.6013 - val_loss: 0.6630 - val_accuracy:

In [47]:
Prediction6 = model6.predict(X_test)

In [48]:
prediction6 = []
for i in Prediction6:
  if i<0.5:
    prediction6.append(0)
  else:
    prediction6.append(1)

score=accuracy_score(y_test,prediction6)
print(score)
print(cr(y_test,prediction6))

0.59735807199758
              precision    recall  f1-score   support

       False       0.63      0.45      0.52      4909
        True       0.58      0.75      0.65      5008

    accuracy                           0.60      9917
   macro avg       0.61      0.60      0.59      9917
weighted avg       0.61      0.60      0.59      9917

